In [85]:
#!pip install sentence-transformers

In [86]:
# standard imports
import urllib.request
import logging
import ssl
from urllib.request import quote,unquote
import requests
import joblib
import os
import re
import warnings
import json
import time
import argparse
import numpy as np
import openai
import tiktoken
import langchain

import pandas as pd
import concurrent.futures
from langchain.schema import Document
from langchain.callbacks import get_openai_callback
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
import random
# external imports
import faiss
from datetime import datetime
from sentence_transformers import SentenceTransformer


In [87]:
from azureml.core import Workspace
 
ws = Workspace.from_config()
default_keyvault = ws.get_default_keyvault()



In [88]:
ullm_gpt4_key = default_keyvault.get_secret("etg-aips-test-assist-ullm-gpt-4-api-key")
ullm_gpt35_key = default_keyvault.get_secret("etg-aips-test-assist-ullm-gpt-35-turbo-api-key")
ullm_gpt4_url = default_keyvault.get_secret("etg-aips-test-assist-ullm-gpt-4-url")
ullm_gpt35_url = default_keyvault.get_secret("etg-aips-test-assist-ullm-gpt-35-turbo-url")

In [89]:
from azureml.core.model import Model
from azureml.core import Workspace
model_name = "test-assist-code-search-faiss-index"
ws=Workspace.from_config()
#model_path = Model(workspace=ws, name=model_name)
# global model_path
model_path = Model.get_model_path(_workspace=ws,model_name=model_name)

In [90]:
print(model_path)

azureml-models/test-assist-code-search-faiss-index/2/code_to_code_search_vdb


In [91]:
class Inference:

    def __init__(self, model_path):
        self.model_path = model_path
        self.embeddings = SentenceTransformerEmbeddings(
            model_name="multi-qa-MiniLM-L6-cos-v1")
        self.db = FAISS.load_local(self.model_path, self.embeddings,allow_dangerous_deserialization=True)
        self.gpt_35_api_key = ullm_gpt35_key
        self.gpt4_api_key =ullm_gpt4_key
        self.url = "https://apidev.servicenow.com/ullm_dev/ullm-router/score"
 

    def gpt_35_turbo_16k_chat_completion(self,prompt):
        #self.allowSelfSignedHttps(True)
        print(prompt)
        value = [{"role": "user", "content": quote(prompt)}]
        value_unquote = [{"role": "user", "content": prompt}]
        turbo_data = {
            "messages": value,
            'temperature': 0,
            'top_p': 0.1
        }

        turbo_data_unquote = {
            "messages": value_unquote,
            'temperature': 0,
            'top_p': 0.1
        }
        print(str(turbo_data_unquote))
        #print(str(turbo_data))
        turbo_body = str.encode(json.dumps(turbo_data))
        url = self.url
        api_key = self.gpt_35_api_key
        api_key_gpt4_32k = self.gpt4_api_key
        if not api_key:
            raise Exception("A key should be provided to invoke the endpoint")

        headers = {'Content-Type': 'application/json','api-key': api_key}

        #req = urllib.request.Request(url, turbo_body, headers)
        result =""
        prompt_response=""
        tokens_count = 0
        total_cost = 0.0
        model =""
        status =""
        status_code = 0
        error_message=""

        try:
            #req = urllib.request.Request(url, turbo_body, headers)
            response = requests.post(url=url,headers=headers,data=turbo_body)
            status_code = response.status_code
            response_json = response.json()
            print(response_json)
            prompt_response = response_json['choices'][0]['message']['content']
            print(prompt_response)
            tokens_count = response_json['usage']['total_tokens']
            total_cost = (int(response_json['usage']['prompt_tokens'])/1000)*0.0005 + (int(response_json['usage']['completion_tokens'])/1000)*0.0015
            total_cost = '{:.8f}'.format(total_cost)
            model = response_json['model']
            status = "success"

        except requests.exceptions.RequestException as error:  # This is the correct syntax
            
            status_code = error.errno
            status = "failed"
            print(error)
            #logger.info(error_message)
            raise(error)

        return prompt_response,tokens_count,total_cost,model,status,status_code,error_message

    def num_tokens_from_string(self, string: str, encoding_name: str) -> int:
        encoding = tiktoken.get_encoding(encoding_name)
        num_tokens = len(encoding.encode(string))
        return num_tokens

    def code_clean(self,code):
        code = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b', f"{{{'email'}}}", code)
        code = re.sub(r'\b\d{3}[-.]?\d{3}[-.]?\d{4}\b', f"{{{'phone'}}}", code)
        code = re.sub(r'\b\d{3}[-]?\d{2}[-]?\d{4}\b', f"{{{'ssn '}}}", code)
        code = re.sub(r'\b\d{3}[ ]?\d{2}[ ]?\d{4}\b', f"{{{'ssn'}}}", code)
        code = re.sub(r'\b(?:\d{1,3}\.){3}\d{1,3}\b', f"{{{'ip address'}}}", code)
        code = re.sub(r'//.*', '', code)
        code = re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL)
        return code

    # Without gloobal variables
    def split_func(self,code):
        code = self.code_clean(code)
        classes=code.split('\n')
        classlist=[]
        scriptlist=[]
        cla=''
        for i in range(len(classes)):
            #dividing classes
            if len(re.findall(r'Class\.create\(\)',classes[i]))>0:
                classlist.append(cla)
                cla=''
                cla+=classes[i]+'\n'
            else:
                cla+=classes[i]+'\n'
            if(i==len(classes)-1):
                classlist.append(cla)
                cla=''
        classlist.pop(0)
        final_classlist=[]
        for i in range(len(classlist)):   
            #counting number of functions in every class
            fu=re.findall(r':\s*function\s*\(',classlist[i])
            if(len(fu)>1):
                lines=classlist[i].split('\n')
                #getting all the script before starting first function
                prefix=''
                ind=2
                for j in range(len(lines)):
                    if len(re.findall(r':\s*function\s*\(',lines[j]))>0:
                        ind=j
                        break
                    else:
                        prefix+='\n'+lines[j]
                #not considering global variables
                temp_prefix=prefix.split('\n')
                prefix2=temp_prefix[1]+'\n'+temp_prefix[-1]
            
                #exluding class defining code lines
                nlines=lines[ind:-2]
                func=[]
                temp=''
                br=0
                comment=False
                for j in range(len(nlines)):
                    if len(re.findall(r':\s*function\s*\(',nlines[j]))>0 and '{' in nlines[j]:
                        br+=1
                        temp+='\n'+nlines[j]
                        continue
                    elif len(re.findall(r':\s*function\s*\(',nlines[j]))>0 and '{' not in nlines[j]:
                        temp+='\n'+nlines[j]
                        continue
                    if '{' in nlines[j]:
                        br+=1
                    if '}' in nlines[j]:
                        br-=1
                    if br==0 and '}' in nlines[j]:
                        temp+='\n'+nlines[j]
                        #adding class defining code prior to function
                        temp=prefix2+temp+'\n'+'};'
                        #appending all functions snippets to list
                        func.append(temp)
                        temp=''
                    if len(temp)>0:
                        temp+='\n'+nlines[j]
                    
                #replacing that paticular class string with list of function strings
                for script in func:
                    final_classlist.append(script)
            else:
                final_classlist.append(classlist[i])

        return final_classlist

    def gpt35turbo_large_script_generate(self,code):
        token_count = self.num_tokens_from_string(code,"cl100k_base")
        no_list = int(np.ceil(token_count/4000))
        list_func = self.split_func(code)
        # using list comprehension 
        num_rec = int(np.ceil(len(list_func)/no_list))
        list_func_split = [list_func[i:i + num_rec] for i in range(0, len(list_func), num_rec)]
        final_list=[]
        for i in list_func_split:
            if self.num_tokens_from_string('\n'.join(i),"cl100k_base")>6000:
                token_count_s = self.num_tokens_from_string('\n'.join(i),"cl100k_base")
                no_list_s = int(np.ceil(token_count_s/4000))
                list_func_s = self.split_func('\n'.join(i))
                # using list comprehension 
                num_rec_s = int(np.ceil(len(list_func_s)/no_list_s))
                list_func_split_s = [list_func_s[i:i + num_rec_s] for i in range(0, len(list_func_s), num_rec_s)]
                for j in list_func_split_s:
                    final_list.append('\n'.join(j))
            else:
                final_list.append('\n'.join(i))

        response,time_taken,token_count,total_cost,model_used,error,error_type,azure_time_taken,self_calculated_token_size = self.parallel_api_call(final_list)
        
        return response,time_taken,token_count,total_cost,model_used,error,error_type,azure_time_taken,self_calculated_token_size

    def restructure_document_object(self, matching_docs):
        filtered_documents = []
        for row in matching_docs:
            input_doc_content = row.page_content + \
                "\n\n" + row.metadata.get('unit_test_cases')
            document_obj = Document(page_content=input_doc_content)
            filtered_documents.append(document_obj)
        return filtered_documents

    def gpt35turbo_langchain_generate(self, code):

        start_time = datetime.now()
        response = None
        total_cost = ""
        tokens_count = ""
        model_used = ""
        error = ""
        error_type = ""
        azure_start_time = ""
        azure_end_time = ""
        input_prompt_size=0
        matching_docs=[]
        if len(code) <= 5:
            response = None
            error = "length of source code is too short to generate a response"
            error_type = "not_enough_tokens"

        elif len(code) > 5:

            code = self.code_clean(code)
            prompt = '''
            Task:
            write unit test cases in Jasmine Framework for below servicenow script, create various test scenarios using multiple data points

            Instructions:
            - understand the code snippet below and write unit test cases
            - consider every scenario for unit testing
            - only write code
            - no comments and explanations for the written code

            code snippet:
            ####

            {0}

            ####

            Example output format:
            //comments
            <code>
            </code>
            '''.format(code)

            input_prompt_size = self.num_tokens_from_string(
                prompt, "cl100k_base")
            print("input prompt size: " + str(input_prompt_size))

            if input_prompt_size < 1500:
                model_used = 'gpt-35-turbo'
                #llm = self.llm_gpt35_4k

                retriever = self.db.as_retriever(
                    search_type="similarity_score_threshold", search_kwargs={
                        "score_threshold": 0.5, "k": 1})
                matching_docs = retriever.get_relevant_documents(code)

                # if len(matching_docs) == 0:
                #    retriever = self.db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5,"k": 1})
                #    matching_docs = retriever.get_relevant_documents(code)

            elif input_prompt_size >= 1500 and input_prompt_size < 6000:
                model_used = 'gpt-35-turbo-16k'
                #llm = self.llm_gpt35_16k

                retriever = self.db.as_retriever(
                    search_type="similarity_score_threshold", search_kwargs={
                        "score_threshold": 0.5, "k": 1})
                matching_docs = retriever.get_relevant_documents(code)

            else:
                model_used = 'gpt-35-turbo-16k'
                #llm = self.llm_gpt35_16k
                matching_docs = []

            azure_start_time = datetime.now()

            if len(matching_docs) == 0:

                    #response = llm(messages=[HumanMessage(content=prompt)])
                prompt_response,tokens_count,total_cost,model,status,status_code,error_message = self.gpt_35_turbo_16k_chat_completion(prompt)
            else:
                matching_docs = self.restructure_document_object(
                    matching_docs)
                chain = load_qa_chain(self.gpt_35_turbo_16k_chat_completion, chain_type="stuff")
                response,tokens_count,total_cost,model,status,status_code,error  = chain.run(
                    input_documents=matching_docs, question=prompt)
        azure_end_time = datetime.now()

        # md_token_count = 0
        # if len(matching_docs) !=0:
        #     for i in matching_docs:
        #         md_token_count = md_token_count + self.num_tokens_from_string(i.page_content,"cl100k_base")

        # self_calculated_token_size = input_prompt_size + self.num_tokens_from_string(response,"cl100k_base") + md_token_count
        end_time = datetime.now()
        azure_time_taken = round(
            (azure_end_time - azure_start_time).total_seconds(),
            2)  # type: ignore
        time_taken = round((end_time - start_time).total_seconds(), 2)
        return response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken

    
    def parallel_api_call(self,final_list):
        # Define the number of threads (adjust as needed)
        num_threads = len(final_list)
        
        with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
            # Submit tasks to the executor
            futures = [executor.submit(self.gpt35turbo_langchain_generate, item) for item in final_list]
        
            # Wait for all tasks to complete
            concurrent.futures.wait(futures)
        
            # Get results
            results = [future.result() for future in futures]
            response = '\n'.join([x[0] for x in results])
            time_taken = max([x[1] for x in results])
            token_count = sum([x[2] for x in results])
            total_cost = sum([x[3] for x in results])
            model_used = ",".join(list(set([x[4] for x in results])))
            error = ",".join(list(set([x[5] for x in results])))
            error_type = ",".join(list(set([x[6] for x in results])))
            azure_time_taken = max([x[7] for x in results])
            self_calculated_token_size = sum([x[8] for x in results])
        return response,time_taken,token_count,total_cost,model_used,error,error_type,azure_time_taken,self_calculated_token_size



In [92]:
small_code = '''var ImpactSPMIntegrationService = Class.create();

ImpactSPMIntegrationService.prototype = {
    // Initialize tables used in integration
    initialize: function() {
        this.SPM_ENTITY_LINKAGE_DETAIL_TABLE = "x_snc_impact_spm_entity_linkage_detail";
        this.SPM_ENTITY_DETAIL_TABLE = "x_snc_impact_spm_entity_detail";
        this.SPM_ENTITY_TABLE = "x_snc_impact_spm_entity";
        this.SPM_INTEGRATION_SETUP_TABLE = "x_snc_impact_spm_integration_setup";
        this.CUSTOMER_CONTACTS_SPM_INTEGRATION_SETUP_RELATION_TABLE = "x_snc_impact_spm_m2m_x_snc_cust_c_x_snc_impac";
    }'''

In [93]:
code = '''
var ImpactSPMIntegrationService = Class.create();

ImpactSPMIntegrationService.prototype = {
    // Initialize tables used in integration
    initialize: function() {
        this.SPM_ENTITY_LINKAGE_DETAIL_TABLE = "x_snc_impact_spm_entity_linkage_detail";
        this.SPM_ENTITY_DETAIL_TABLE = "x_snc_impact_spm_entity_detail";
        this.SPM_ENTITY_TABLE = "x_snc_impact_spm_entity";
        this.SPM_INTEGRATION_SETUP_TABLE = "x_snc_impact_spm_integration_setup";
        this.CUSTOMER_CONTACTS_SPM_INTEGRATION_SETUP_RELATION_TABLE = "x_snc_impact_spm_m2m_x_snc_cust_c_x_snc_impac";
    },

    /**
     * Returns a filtered list of SPM Entities
     *
     * @param {String} companyId - Sys id of the company
     * @param {String} tableName - Table name of spm entity
     * @param {String} instanceId - SPM table sys id
     * @param {String} instanceName - SPM table name
     * @param {Number} pageNo - SPM link table pagination number
     * @param {Array<{field: string, operator: string, value: string}>} filters - Object containing the filter criteria
     * @returns {Array<Object>} An array of SPM Entities that match the filter criteria
     */
    getFilteredSPMEntities: function(companyId, tableName, instanceId, instanceName, pageObj, filters) {
        try {
            var spmSetupService = new x_snc_impact_spm.ImpactSPMSetupService();
            var spmImpactKeyMap = spmSetupService.getSPMImpactMapping(companyId, instanceId, tableName);
            filters.forEach(function(filter) {
                filter.field = spmImpactKeyMap[filter.field] || filter.field;
            });
            filters.push({
                field: '',
                value: spmImpactKeyMap.spm_name,
                operator: 'ORDERBY',
            });
            var spmResponse = spmSetupService.fetchTableData(
                companyId, {
                    instanceName: instanceName,
                    instanceId: instanceId
                },
                tableName,
                pageObj,
                filters,
            );
            return spmResponse;
        } catch (ex) {
            gs.error('Exception in getFilteredSPMEntities: ' + ex.stack);
        }
    },
    /**
     * Save the SPM Entity in Impact and link both entities
     * @param {string} companyId - Sys ID of the company
     * @param instanceObj {json object} - Instance object containing instance ID and instance name
     * @param {Object} impactEntity - Impact Entity to be Linked.
     * @param {Object} spmEntities - List of SPM Entities to save and Link.
     * @param {Object} toBeDeletedSysIds - List of spm record sys ids of linked records to be deleted.
     */
    saveSPMEntities: function(companyId, instanceObj, impactEntity, spmEntities, toBeDeletedSysIds) {

        var instanceRecordId = instanceObj.instanceRecordId;
        var instanceName = instanceObj.instanceName;
        // Delete requested records from link table.
        var deletingRecordSysIds = toBeDeletedSysIds.join(',');
        if (deletingRecordSysIds) {
            this._deleteLinkedRecords(impactEntity, deletingRecordSysIds, companyId);
        }
        // Save and Link records.
        for (var i = 0; i < spmEntities.length; i++) {

            var spmEntityDetail = spmEntities[i];
            var newCreatedRecordSysId, spmDetailRecordSysId;
            var updatedRecordSysId = this._updateSPMDetail(spmEntityDetail, companyId);

            if (!updatedRecordSysId) {
                var entityDetailGR = new GlideRecord(this.SPM_ENTITY_DETAIL_TABLE);

                // Get SPM Table SysId for the SPM Table.
                var spmEntityGR = new GlideRecord(this.SPM_ENTITY_TABLE);
                var spmTableId;
                var spmTableName = spmEntityDetail.spm_table_detail;
                spmEntityGR.addQuery('table_detail', spmTableName);
                spmEntityGR.query();
                if (spmEntityGR.next() && spmEntityGR.canRead()) {
                    spmTableId = spmEntityGR.getUniqueValue();
                }


                entityDetailGR.initialize();
                entityDetailGR.setValue('company', companyId);
                entityDetailGR.setValue('instance', instanceRecordId);
                entityDetailGR.setValue('spm_table', spmTableId);

                for (var prop in spmEntityDetail) {
                    if (!entityDetailGR[prop]) {
                        entityDetailGR[prop] = spmEntityDetail[prop];
                    }
                }
                var spm_record_url = gs.getMessage(gs.getProperty('x_snc_impact_spm.record.url'), [instanceName, spmTableName, spmEntityDetail.spm_record_id]);
                entityDetailGR.setValue('spm_record_url', spm_record_url);
                if (entityDetailGR.canCreate()) {
                    newCreatedRecordSysId = entityDetailGR.insert();
                }
            }
            spmDetailRecordSysId = updatedRecordSysId ? updatedRecordSysId : newCreatedRecordSysId;
            if (!this._recordsAlreadyLinked(spmDetailRecordSysId, impactEntity, companyId)) {
                this._linkImpactSPMEntities(companyId, spmDetailRecordSysId, impactEntity);
            }
        }
    },
    /**
     * Delete linked records
     * @param {object} impactEntity - Impact entity on which linking/deleting is happening. 
     * @param {string} deletingSPMRecordSysIds - Comma seperated sys-ids of records to be deleted.
	 * @param {string} companyId - SysId of the company
     */
    _deleteLinkedRecords(impactEntity, deletingSPMRecordSysIds, companyId) {

        var linkEntityGR = new GlideRecord(this.SPM_ENTITY_LINKAGE_DETAIL_TABLE);
        // Deleting records.
        linkEntityGR.addQuery('impact_entity_record', impactEntity.sys_id);
		linkEntityGR.addQuery('company', companyId);
        linkEntityGR.addQuery('spm_entity_record.spm_record_id', 'IN', deletingSPMRecordSysIds);
        linkEntityGR.query();
        while (linkEntityGR.next() && linkEntityGR.canDelete()) {
            linkEntityGR.deleteRecord();
        }
    },
    /**
     * Link Entity in Impact with SPM entity
     * @param {string} spmDetailRecordSysId - Sys Id of created SPM record
     * @param {Object} impactEntity - Impact Entity to be Linked {tablename, record sys id}.
	 * @param {string} companyId - SysId of the company
     * @returns {Boolean} If both records are already linked or not.
     */
    _recordsAlreadyLinked(spmDetailRecordSysId, impactEntity, companyId) {
        var linkEntityGR = new GlideRecord(this.SPM_ENTITY_LINKAGE_DETAIL_TABLE);
        linkEntityGR.addQuery('impact_entity_record', impactEntity.sys_id);
        linkEntityGR.addQuery('spm_entity_record', spmDetailRecordSysId);
		linkEntityGR.addQuery('company', companyId);
        linkEntityGR.query();
        if (linkEntityGR.next() && linkEntityGR.canRead()) {
            return true;
        }
        return false;
    },
    /**
     * Link Entity in Impact with SPM entity
     * @param {string} companyId - Sys Id of company
     * @param {string} spmDetailRecordSysId - Sys Id of created SPM record
     * @param {Object} impactEntity - Impact Entity to be Linked {tablename, record sys id}.
     * @param {Object} toBeDeletedSysIds - List of sysIds of linked records to be deleted.
     */
    _linkImpactSPMEntities(companyId, spmDetailRecordSysId, impactEntity) {

        var impactEntitySysId = impactEntity.sys_id;
        var impactEntityRecordTable = impactEntity.table_detail;
        var linkEntityGR = new GlideRecord(this.SPM_ENTITY_LINKAGE_DETAIL_TABLE);
        // Link newly created/updated record.

        linkEntityGR.initialize();
        linkEntityGR.setValue('company', companyId);
        linkEntityGR.setValue('impact_entity_record', impactEntitySysId);
        linkEntityGR.setValue('impact_entity_record_table', impactEntityRecordTable);
        linkEntityGR.setValue('spm_entity_record', spmDetailRecordSysId);
        if (linkEntityGR.canCreate()) {
            linkEntityGR.insert();
        }
    },

    /**
     * Sort an Array of Object by name
     * @param {Array} list - An array of objects with name filed
     * @returns {Array} An array of object sorted by name
     */
    _sortByName: function(list) {
        return list.sort(function(a, b) {
            const valueA = a.name.toLowerCase();
            const valueB = b.name.toLowerCase();
            return valueA < valueB ? -1 : (valueA > valueB ? 1 : 0);
        });
    },

    /**
     * Retrieves the set of linked SPM Entities for an impact record
     * @param {string} impactEntityId - impact entity record
     * @returns {Array} An array of SPM Entities that are linked to the impact record
     */
    # getLinkedSPMEntities: function(impactEntityId, companyId) {
    #     var gr = new GlideRecord(this.SPM_ENTITY_LINKAGE_DETAIL_TABLE);
    #     gr.addQuery('impact_entity_record', impactEntityId);
    #     gr.addQuery('company', companyId);
    #     gr.query();
    #     var linkedSPMEntities = {};
    #     while (gr.next() && gr.canRead()) {
    #         var spmRecordId = gr.spm_entity_record.spm_record_id.toString();
    #         var spmTable = gr.spm_entity_record.spm_table.table_detail;
    #         if (!linkedSPMEntities.hasOwnProperty(spmTable)) {
    #             linkedSPMEntities[spmTable] = [];
    #         }
    #         linkedSPMEntities[spmTable].push(spmRecordId);
    #     }
    #     return linkedSPMEntities;
    # },

    /**
     * Determine whether the current entity linked with any spm record
     * @param {string} impactEntityId - impact entity record
     * @param {string} companyId - Sys ID of the account
     * @returns {Object} if linked to the impact record for that specific company, false if not
     */
    hasLinkedSPMEntities: function(impactEntityId, companyId) {
        var linkageGr = new GlideRecord(this.SPM_ENTITY_LINKAGE_DETAIL_TABLE);
        linkageGr.addQuery('impact_entity_record', impactEntityId);
        linkageGr.addQuery('company', companyId);
        linkageGr.addQuery('company.x_snc_account_cust_spm_integration_enabled', true);
        linkageGr.query();
        if (linkageGr.next() && linkageGr.canRead()) {
            return {
                showLinkedSPMButton: true,
                spmInstanceName: linkageGr.getDisplayValue('spm_entity_record.instance'),
            };
        }
        return {
            showLinkedSPMButton: false,
            spmInstanceName: '',
        };
    },

    /**
     * Determines whether the current user has permission to link/unlink SPM Entities
     *
     * @param {string} companyId - Sys ID of the account
     * @param {string} instanceId - Instance ID
     * @param {string} currentUser - Sys ID of the current user
     * @returns {Boolean} True if the user has permission, false if not
     */
    hasLinkingAccess: function(companyId, instanceId, currentUser) {
        var gr = new GlideRecord(this.CUSTOMER_CONTACTS_SPM_INTEGRATION_SETUP_RELATION_TABLE);
        gr.addQuery('x_snc_impact_spm_integration_setup.account', companyId);
        gr.addQuery('x_snc_impact_spm_integration_setup.instance', instanceId);
        gr.addQuery('x_snc_cust_contact_customer_contacts.user', currentUser);
        gr.query();
        return gr.next() && gr.canRead();
    },

    /**
     * saves the set of SPM entity details from the external data source
     * @param {Object} spmEntities - An Object of SPM entity detail that are linked to the impact record
	 * @param {String} companyId - SysId of the Company
     */
    # _updateSPMDetail: function(entityDetail, companyId) {
    #     var entityDetailRecord = new GlideRecord(this.SPM_ENTITY_DETAIL_TABLE);
    #     var spmRecordId = entityDetail.spm_record_id;
	# 	entityDetailRecord.addQuery('spm_record_id', spmRecordId);
    #     entityDetailRecord.addQuery('company', companyId);
    #     entityDetailRecord.query();
    #     if (entityDetailRecord.next() && entityDetailRecord.canWrite()) {
    #         entityDetailRecord.setValue('name', entityDetail.name);
    #         entityDetailRecord.setValue('status', entityDetail.status);
    #         entityDetailRecord.setValue('state', entityDetail.state);
    #         entityDetailRecord.setValue('short_description', entityDetail.short_description);
    #         entityDetailRecord.setValue('start_date', entityDetail.start_date);
    #         entityDetailRecord.setValue('end_date', entityDetail.end_date);
    #         return entityDetailRecord.update();
    #     }
    #     return null;
    # },
    /**
     * Refreshes the set of SPM Entities from the external data source
     * @param {string} companyId - Sys ID of the account
     * @param {string} instanceId - Instance ID
     * @param {string} instanceName - Instance Name
     * @param {string} impactEntityId - Impact Entity Linked
     * @returns {Array} An array of SPM Entities details that are linked to the impact record
     */
    refreshSPMEntities: function(companyId, instanceId, instanceName, impactEntityId) {
        try {
            var instanceObj = {
                instanceId: instanceId,
                instanceName: instanceName
            };
            var linkedSPMEntities = this.getLinkedSPMEntities(impactEntityId, companyId);
            var pageNo = 0;
            var response = {
                data: [],
                status: "",
            };
            var entitiesData = {},
                spmEntitiesIds = "";
            for (var tableName in linkedSPMEntities) {
                var filters = [{
                    field: "sys_id",
                    operator: "IN",
                    value: linkedSPMEntities[tableName].join(",")
                }];
                spmEntitiesIds += spmEntitiesIds ? "," + filters[0].value : filters[0].value;
                var entityDetails = new x_snc_impact_spm.ImpactSPMSetupService().fetchTableData(
                    companyId,
                    instanceObj,
                    tableName,
                    pageNo,
                    filters,
                );
                if (entityDetails.status === "success") {
                    response.status = "success";
                    entityDetails.data.forEach(function(entity) {
                        entity.record_not_found = "false";
                        entitiesData[entity["spm_record_id"]] = entity;
                    });

                } else if (entityDetails.httpStatus == 0) {
                    response.status = "error";
                    return response;
                }

                linkedSPMEntities[tableName].forEach(function(entity) {
                    if (!(entity in entitiesData)) {
                        var entityData = {};
                        entityData.spm_record_id = entity;
                        entityData.record_not_found = "true";
                        entitiesData[entity] = entityData;
                    }
                });
            }
            var result = this._updateSPMEntitiesDetails(spmEntitiesIds, entitiesData, companyId);
            response.data.push.apply(response.data, result);
            return response;
        } catch (ex) {
            gs.error('Exception in refreshSPMEntities: ' + ex.stack);
        }
    },
    # _updateSPMEntitiesDetails: function(spmEntitiesIds, spmEntitiesDetails, companyId) {
    #     var entityDetailRecord = new GlideRecord(this.SPM_ENTITY_DETAIL_TABLE);
    #     entityDetailRecord.addEncodedQuery('spm_record_idIN' + spmEntitiesIds);
	# 	entityDetailRecord.addQuery('company', companyId);
    #     entityDetailRecord.orderBy('name');
    #     entityDetailRecord.query();
    #     var result = [];
    #     while (entityDetailRecord.next() && entityDetailRecord.canWrite()) {
    #         var entityDetail = spmEntitiesDetails[entityDetailRecord.getValue("spm_record_id")];
    #         if (entityDetail.record_not_found == "true") {
    #             entityDetailRecord.setValue('status', 'NULL');
    #             entityDetailRecord.setValue('state', 'NULL');
    #             entityDetailRecord.start_date = 'NULL';
    #             entityDetailRecord.end_date = 'NULL';
    #             entityDetailRecord.setValue('record_not_found', true);
    #             entityDetail.name = entityDetailRecord.getValue('name');
    #             entityDetail.short_description = entityDetailRecord.getValue('short_description');
    #             entityDetail.spm_table = entityDetailRecord.getDisplayValue('spm_table');
    #             entityDetail.spm_record_url = entityDetailRecord.getDisplayValue('spm_record_url');
    #             entityDetail.buttonText = gs.getMessage('impact-spm-entity-button', entityDetail.spm_table.toLowerCase());
    #             entityDetail.status = "NA";
    #             entityDetail.state = "NA";
    #             entityDetail.start_date = "NA";
    #             entityDetail.end_date = "NA";
    #         } else {
    #             entityDetailRecord.setValue('name', entityDetail.name);
    #             entityDetailRecord.setValue('status', entityDetail.status);
    #             entityDetailRecord.setValue('state', entityDetail.state);
    #             entityDetailRecord.setValue('short_description', entityDetail.short_description);
    #             entityDetailRecord.setValue('start_date', entityDetail.start_date);
    #             entityDetailRecord.setValue('end_date', entityDetail.end_date);
    #             entityDetailRecord.setValue('record_not_found', false);
    #         }
    #         entityDetail.buttonText = gs.getMessage('impact-spm-entity-button', entityDetailRecord.getDisplayValue('spm_table').toLowerCase());
    #         entityDetailRecord.update();
    #         result.push(entityDetail);
    #     }
    #     return result;
    # },




    /**
     * Deletes any unlinked SPM Entities from the system in batch.
     */
    deleteUnlinkedSPMEntities: function() {
        var detailCountGR = new GlideRecord(this.SPM_ENTITY_DETAIL_TABLE);
        detailCountGR.query();
        var totalCount = detailCountGR.getRowCount();

        var start = 0,
            end = 200;
        var detailGR = new GlideRecord(this.SPM_ENTITY_DETAIL_TABLE);

        while (start <= totalCount) {
            detailGR.chooseWindow(start, end);
            detailGR.query();
            while (detailGR.next() && detailGR.canDelete()) {
                var recordSysId = detailGR.getUniqueValue();
                var linkGR = new GlideRecord(this.SPM_ENTITY_LINKAGE_DETAIL_TABLE);
                linkGR.addQuery('spm_entity_record', recordSysId);
                linkGR.query();
                if (!linkGR.next()) {
                    detailGR.deleteRecord();
                }
            }
            // Updating Window
            start = end;
            end = end + 200;
            this._sleep(5000);
        }
    },
    /*
     * Mocks sleep.
     */
    _sleep: function(ms) {
        var unixtime_ms = new Date().getTime();
        while (new Date().getTime() < unixtime_ms + ms) {}
    },
    /**
     * Returns the SPM instance Name and instance Id
     * @param {string} companyId - Sys ID of the account
     * @returns {Object} instanceObj - Object containing instanceName and instanceId
     */
    getSPMInstanceData: function(companyId) {
        var instanceObj = {};
        var spmSetupGr = new GlideRecord('x_snc_impact_spm_integration_setup');
        spmSetupGr.addQuery('account', companyId);
        spmSetupGr.setLimit(1);
        spmSetupGr.query();
        if (spmSetupGr.next() && spmSetupGr.canRead()) {
            instanceObj = {
                instanceName: spmSetupGr.getDisplayValue('instance.snc_instance_name'),
                instanceId: spmSetupGr.getDisplayValue('instance.snc_instance_id'),
                instanceRecordId: spmSetupGr.getValue('instance')
            };
        }
        return instanceObj;
    },

    /**
     * Raju S
     * Returns the set of linked SPM Entities for an impact record
     * @param {string} impactEntityId - impact entity record
     * @returns {Array} An array of SPM Entities that are linked to the impact record
     */
    getLinkedSPMEntityData: function(companyId, impactEntitySysId, pageObj) {
        var pagination = pageObj.pageNo && pageObj.itemsPerPage;
        var grLinkage = new GlideRecord(this.SPM_ENTITY_LINKAGE_DETAIL_TABLE);
        var query = 'impact_entity_record=' + impactEntitySysId + '^company=' + companyId;

        grLinkage.addEncodedQuery(query);
        grLinkage.orderBy('spm_entity_record.name');
        if (pagination) {
            grLinkage.chooseWindow(pageObj.pageNo - 1 * pageObj.itemsPerPage, pageObj.pageNo * pageObj.itemsPerPage);
        }
        grLinkage.query();
        var count = grLinkage.getRowCount();
        var result = [];
        while (grLinkage.next() && grLinkage.canRead()) {
            var obj = {};
            obj.linkingId = grLinkage.getValue('impact_entity_record');
            obj.linked_record_sys_id = grLinkage.getUniqueValue();
            obj.spmEntityRecord = grLinkage.getValue('spm_entity_record');

            obj.name = grLinkage.getDisplayValue('spm_entity_record.name');
            obj.short_description = grLinkage.getDisplayValue('spm_entity_record.short_description');

            obj.start_date = grLinkage.spm_entity_record.start_date.toString();
            obj.end_date = grLinkage.spm_entity_record.end_date.toString();

            obj.status = grLinkage.getDisplayValue('spm_entity_record.status');

            obj.state = grLinkage.getDisplayValue('spm_entity_record.state');

            obj.spm_record_id = grLinkage.getDisplayValue('spm_entity_record.spm_record_id');

            obj.spm_table = grLinkage.getDisplayValue('spm_entity_record.spm_table');
            obj.spm_record_url = grLinkage.getDisplayValue('spm_entity_record.spm_record_url');
            obj.buttonText = gs.getMessage('impact-spm-entity-button', obj.spm_table.toLowerCase());
            obj.record_not_found = grLinkage.spm_entity_record.record_not_found.toString();
            result.push(obj);

        }
        return {
            result: result,
            totalRecord: count
        };
    },

    /**
     * This method will return true if the current user is squad and can create/write mapping table for the given company.
     * @param {string} sys id of the company
     * @return {boolean}
     */
	
    canEditFieldMapping: function(companySysId) {
        var squadRole = false;
        if (gs.hasRole("x_snc_cusp.user")) { //internal employees
            var squadGR = new GlideRecord("x_snc_cust_contact_customer_contacts");
            squadGR.addQuery('company', companySysId);
            squadGR.addQuery('user', gs.getUserID()); 
            squadGR.addQuery('contact_role.type', 'internal'); //for only squads (& not extended squads) assigned current company
            squadGR.query();
            squadRole = squadGR.canRead() && squadGR.next();
            return squadRole ;
        }
        return false;
    },

    type: 'ImpactSPMIntegrationService'
};
// End of script
'''

In [94]:
model=Inference(model_path=model_path)

In [95]:
response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_langchain_generate(code)


input prompt size: 4018

            Task:
            write unit test cases in Jasmine Framework for below servicenow script, create various test scenarios using multiple data points

            Instructions:
            - understand the code snippet below and write unit test cases
            - consider every scenario for unit testing
            - only write code
            - no comments and explanations for the written code

            code snippet:
            ####

            
var ImpactSPMIntegrationService = Class.create();

ImpactSPMIntegrationService.prototype = {
    
    initialize: function() {
        this.SPM_ENTITY_LINKAGE_DETAIL_TABLE = "x_snc_impact_spm_entity_linkage_detail";
        this.SPM_ENTITY_DETAIL_TABLE = "x_snc_impact_spm_entity_detail";
        this.SPM_ENTITY_TABLE = "x_snc_impact_spm_entity";
        this.SPM_INTEGRATION_SETUP_TABLE = "x_snc_impact_spm_integration_setup";
        this.CUSTOMER_CONTACTS_SPM_INTEGRATION_SETUP_RELATION_TABLE = "x_snc_im

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/langchain_core/vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.5
  warnings.warn(


In [29]:
def parse_json(data):
    result = {}
    for key, value in data.items():
        if isinstance(value, dict):
            result[key] = parse_json(value)
        else:
            result[key] = value
    return result
 
# Parsing JSON data using recursion


In [36]:
print(json.loads(json.dumps(response)))

{"choices": [{"finish_reason": "stop", "index": 0, "message": {"content": "//Unit test cases for ImpactSPMIntegrationService

describe("ImpactSPMIntegrationService", function() {

  var service;

  beforeEach(function() {
    service = new ImpactSPMIntegrationService();
  });

  it("should initialize variables correctly", function() {
    expect(service.SPM_ENTITY_LINKAGE_DETAIL_TABLE).toEqual("x_snc_impact_spm_entity_linkage_detail");
    expect(service.SPM_ENTITY_DETAIL_TABLE).toEqual("x_snc_impact_spm_entity_detail");
    expect(service.SPM_ENTITY_TABLE).toEqual("x_snc_impact_spm_entity");
    expect(service.SPM_INTEGRATION_SETUP_TABLE).toEqual("x_snc_impact_spm_integration_setup");
    expect(service.CUSTOMER_CONTACTS_SPM_INTEGRATION_SETUP_RELATION_TABLE).toEqual("x_snc_impact_spm_m2m_x_snc_cust_c_x_snc_impac");
  });

  it("should get filtered SPM entities correctly", function() {
    var companyId = "123";
    var tableName = "incident";
    var instanceId = "456";
    var instan

In [30]:
response = '''{"choices": [{"finish_reason": "stop", "index": 0, "message": {"content": "//Unit test cases for ImpactSPMIntegrationService\n\ndescribe(\"ImpactSPMIntegrationService\", function() {\n\n  var service;\n\n  beforeEach(function() {\n    service = new ImpactSPMIntegrationService();\n  });\n\n  it(\"should initialize variables correctly\", function() {\n    expect(service.SPM_ENTITY_LINKAGE_DETAIL_TABLE).toEqual(\"x_snc_impact_spm_entity_linkage_detail\");\n    expect(service.SPM_ENTITY_DETAIL_TABLE).toEqual(\"x_snc_impact_spm_entity_detail\");\n    expect(service.SPM_ENTITY_TABLE).toEqual(\"x_snc_impact_spm_entity\");\n    expect(service.SPM_INTEGRATION_SETUP_TABLE).toEqual(\"x_snc_impact_spm_integration_setup\");\n    expect(service.CUSTOMER_CONTACTS_SPM_INTEGRATION_SETUP_RELATION_TABLE).toEqual(\"x_snc_impact_spm_m2m_x_snc_cust_c_x_snc_impac\");\n  });\n\n  it(\"should get filtered SPM entities correctly\", function() {\n    var companyId = \"123\";\n    var tableName = \"incident\";\n    var instanceId = \"456\";\n    var instanceName = \"testInstance\";\n    var pageObj = {pageNo: 1, itemsPerPage: 10};\n    var filters = [{field: \"short_description\", value: \"test\", operator: \"LIKE\"}];\n    spyOn(x_snc_impact_spm.ImpactSPMSetupService.prototype, \"getSPMImpactMapping\").and.returnValue({short_description: \"description\"});\n    spyOn(x_snc_impact_spm.ImpactSPMSetupService.prototype, \"fetchTableData\").and.returnValue({status: \"success\", data: []});\n    var result = service.getFilteredSPMEntities(companyId, tableName, instanceId, instanceName, pageObj, filters);\n    expect(x_snc_impact_spm.ImpactSPMSetupService.prototype.getSPMImpactMapping).toHaveBeenCalledWith(companyId, instanceId, tableName);\n    expect(x_snc_impact_spm.ImpactSPMSetupService.prototype.fetchTableData).toHaveBeenCalledWith(companyId, {instanceName: instanceName, instanceId: instanceId}, tableName, pageObj, [{field: \"short_description\", value: \"test\", operator: \"LIKE\", field: \"description\"}]);\n    expect(result.status).toEqual(\"success\");\n    expect(result.data).toEqual([]);\n  });\n\n  it(\"should save SPM entities correctly\", function() {\n    var companyId = \"123\";\n    var instanceObj = {instanceRecordId: \"456\", instanceName: \"testInstance\"};\n    var impactEntity = {sys_id: \"789\", table_detail: \"incident\"};\n    var spmEntities = [{name: \"test\", spm_table_detail: \"incident\", spm_record_id: \"111\"}];\n    var toBeDeletedSysIds = [\"222\", \"333\"];\n    spyOn(service, \"_deleteLinkedRecords\");\n    spyOn(service, \"_updateSPMDetail\").and.returnValue(null);\n    spyOn(GlideRecord.prototype, \"initialize\");\n    spyOn(GlideRecord.prototype, \"setValue\");\n    spyOn(GlideRecord.prototype, \"canCreate\").and.returnValue(true);\n    spyOn(GlideRecord.prototype, \"insert\").and.returnValue(\"444\");\n    spyOn(service, \"_recordsAlreadyLinked\").and.returnValue(false);\n    spyOn(service, \"_linkImpactSPMEntities\");\n    service.saveSPMEntities(companyId, instanceObj, impactEntity, spmEntities, toBeDeletedSysIds);\n    expect(service._deleteLinkedRecords).toHaveBeenCalledWith(impactEntity, \"222,333\", companyId);\n    expect(service._updateSPMDetail).toHaveBeenCalledWith({name: \"test\", spm_table_detail: \"incident\", spm_record_id: \"111\"}, companyId);\n    expect(GlideRecord.prototype.initialize).toHaveBeenCalled();\n    expect(GlideRecord.prototype.setValue).toHaveBeenCalledWith(\"company\", companyId);\n    expect(GlideRecord.prototype.setValue).toHaveBeenCalledWith(\"instance\", \"456\");\n    expect(GlideRecord.prototype.setValue).toHaveBeenCalledWith(\"spm_table\", \"incident\");\n    expect(GlideRecord.prototype.setValue).toHaveBeenCalledWith(\"name\", \"test\");\n    expect(GlideRecord.prototype.setValue).toHaveBeenCalledWith(\"spm_record_id\", \"111\");\n    expect(GlideRecord.prototype.setValue).toHaveBeenCalledWith(\"spm_record_url\", \"testInstance/incident.do?sys_id=111\");\n    expect(GlideRecord.prototype.canCreate).toHaveBeenCalled();\n    expect(GlideRecord.prototype.insert).toHaveBeenCalled();\n    expect(service._recordsAlreadyLinked).toHaveBeenCalledWith(\"444\", impactEntity, companyId);\n    expect(service._linkImpactSPMEntities).toHaveBeenCalledWith(companyId, \"444\", impactEntity);\n  });\n\n  it(\"should delete linked records correctly\", function() {\n    var impactEntity = {sys_id: \"789\"};\n    var companyId = \"123\";\n    spyOn(GlideRecord.prototype, \"addQuery\");\n    spyOn(GlideRecord.prototype, \"query\");\n    spyOn(GlideRecord.prototype, \"canDelete\").and.returnValue(true);\n    spyOn(GlideRecord.prototype, \"deleteRecord\");\n    service._deleteLinkedRecords(impactEntity, \"111,222\", companyId);\n    expect(GlideRecord.prototype.addQuery).toHaveBeenCalledWith(\"impact_entity_record\", \"789\");\n    expect(GlideRecord.prototype.addQuery).toHaveBeenCalledWith(\"company\", \"123\");\n    expect(GlideRecord.prototype.addQuery).toHaveBeenCalledWith(\"spm_entity_record.spm_record_id\", \"IN\", \"111,222\");\n    expect(GlideRecord.prototype.query).toHaveBeenCalled();\n    expect(GlideRecord.prototype.canDelete).toHaveBeenCalled();\n    expect(GlideRecord.prototype.deleteRecord).toHaveBeenCalledTimes(2);\n  });\n\n  it(\"should check if records are already linked correctly\", function() {\n    var spmDetailRecordSysId = \"111\";\n    var impactEntity = {sys_id: \"789\"};\n    var companyId = \"123\";\n    spyOn(GlideRecord.prototype, \"addQuery\");\n    spyOn(GlideRecord.prototype, \"query\");\n    spyOn(GlideRecord.prototype, \"canRead\").and.returnValue(true);\n    var result = service._recordsAlreadyLinked(spmDetailRecordSysId, impactEntity, companyId);\n    expect(GlideRecord.prototype.addQuery).toHaveBeenCalledWith(\"impact_entity_record\", \"789\");\n    expect(GlideRecord.prototype.addQuery).toHaveBeenCalledWith(\"spm_entity_record\", \"111\");\n    expect(GlideRecord.prototype.addQuery).toHaveBeenCalledWith(\"company\", \"123\");\n    expect(GlideRecord.prototype.query).toHaveBeenCalled();\n    expect(GlideRecord.prototype.canRead).toHaveBeenCalled();\n    expect(result).toEqual(true);\n  });\n\n  it(\"should link ImpactSPM entities correctly\", function() {\n    var companyId = \"123\";\n    var spmDetailRecordSysId = \"111\";\n    var impactEntity = {sys_id: \"789\", table_detail: \"incident\"};\n    spyOn(GlideRecord.prototype, \"initialize\");\n    spyOn(GlideRecord.prototype, \"setValue\");\n    spyOn(GlideRecord.prototype, \"canCreate\").and.returnValue(true);\n    spyOn(GlideRecord.prototype, \"insert\");\n    service._linkImpactSPMEntities(companyId, spmDetailRecordSysId, impactEntity);\n    expect(GlideRecord.prototype.initialize).toHaveBeenCalled();\n    expect(GlideRecord.prototype.setValue).toHaveBeenCalledWith(\"company\", companyId);\n    expect(GlideRecord.prototype.setValue).toHaveBeenCalledWith(\"impact_entity_record\", \"789\");\n    expect(GlideRecord.prototype.setValue).toHaveBeenCalledWith(\"impact_entity_record_table\", \"incident\");\n    expect(GlideRecord.prototype.setValue).toHaveBeenCalledWith(\"spm_entity_record\", \"111\");\n    expect(GlideRecord.prototype.canCreate).toHaveBeenCalled();\n    expect(GlideRecord.prototype.insert).toHaveBeenCalled();\n  });\n\n  it(\"should sort by name correctly\", function() {\n    var list = [{name: \"test2\"}, {name: \"test1\"}];\n    var result = service._sortByName(list);\n    expect(result).toEqual([{name: \"test1\"}, {name: \"test2\"}]);\n  });\n\n  it(\"should check if linked SPM entities exist correctly\", function() {\n    var impactEntityId = \"123\";\n    var companyId = \"456\";\n    spyOn(GlideRecord.prototype, \"addQuery\");\n    spyOn(GlideRecord.prototype, \"query\");\n    spyOn(GlideRecord.prototype, \"canRead\").and.returnValue(true);\n    var result = service.hasLinkedSPMEntities(impactEntityId, companyId);\n    expect(GlideRecord.prototype.addQuery).toHaveBeenCalledWith(\"impact_entity_record\", \"123\");\n    expect(GlideRecord.prototype.addQuery).toHaveBeenCalledWith(\"company\", \"456\");\n    expect(GlideRecord.prototype.addQuery).toHaveBeenCalledWith(\"company.x_snc_account_cust_spm_integration_enabled\", true);\n    expect(GlideRecord.prototype.query).toHaveBeenCalled();\n    expect(GlideRecord.prototype.canRead).toHaveBeenCalled();\n    expect(result).toEqual({showLinkedSPMButton: true, spmInstanceName: \"\"});\n  });\n\n  it(\"should check if linking access exists correctly\", function() {\n    var companyId = \"123\";\n    var instanceId = \"456\";\n    var currentUser = \"testUser\";\n    spyOn(GlideRecord.prototype, \"addQuery\");\n    spyOn(GlideRecord.prototype, \"query\");\n    spyOn(GlideRecord.prototype, \"next\").and.returnValue(true);\n    spyOn(GlideRecord.prototype, \"canRead\").and.returnValue(true);\n    var result = service.hasLinkingAccess(companyId, instanceId, currentUser);\n    expect(GlideRecord.prototype.addQuery).toHaveBeenCalledWith(\"x_snc_impact_spm_integration_setup.account\", \"123\");\n    expect(GlideRecord.prototype.addQuery).toHaveBeenCalledWith(\"x_snc_impact_spm_integration_setup.instance\", \"456\");\n    expect(GlideRecord.prototype.addQuery).toHaveBeenCalledWith(\"x_snc_cust_contact_customer_contacts.user\", \"testUser\");\n    expect(GlideRecord.prototype.query).toHaveBeenCalled();\n    expect(GlideRecord.prototype.next).toHaveBeenCalled();\n    expect(GlideRecord.prototype.canRead).toHaveBeenCalled();\n    expect(result).toEqual(true);\n  });\n\n  it(\"should refresh SPM entities correctly\", function() {\n    var companyId = \"123\";\n    var instanceId = \"456\";\n    var instanceName = \"testInstance\";\n    var impactEntityId = \"789\";\n    spyOn(service, \"getLinkedSPMEntities\").and.returnValue({result: [], totalRecord: 0});\n    spyOn(service, \"_updateSPMEntitiesDetails\").and.returnValue([]);\n    var result = service.refreshSPMEntities(companyId, instanceId, instanceName, impactEntityId);\n    expect(service.getLinkedSPMEntities).toHaveBeenCalledWith(impactEntityId, companyId);\n    expect(service._updateSPMEntitiesDetails).toHaveBeenCalledWith(\"\", {}, companyId);\n    expect(result.status).toEqual(\"\");\n    expect(result.data).toEqual([]);\n  });\n\n  it(\"should delete unlinked SPM entities correctly\", function() {\n    spyOn(GlideRecord.prototype, \"getRowCount\").and.returnValue(3);\n    spyOn(GlideRecord.prototype, \"chooseWindow\");\n    spyOn(GlideRecord.prototype, \"query\");\n    spyOn(GlideRecord.prototype, \"canDelete\").and.returnValue(true);\n    spyOn(GlideRecord.prototype, \"deleteRecord\");\n    spyOn(service, \"_sleep\");\n    service.deleteUnlinkedSPMEntities();\n    expect(GlideRecord.prototype.getRowCount).toHaveBeenCalled();\n    expect(GlideRecord.prototype.chooseWindow).toHaveBeenCalledTimes(2);\n    expect(GlideRecord.prototype.query).toHaveBeenCalled();\n    expect(GlideRecord.prototype.canDelete).toHaveBeenCalled();\n    expect(GlideRecord.prototype.deleteRecord).toHaveBeenCalledTimes(3);\n    expect(service._sleep).toHaveBeenCalledWith(5000);\n  });\n\n  it(\"should get SPM instance data correctly\", function() {\n    var companyId = \"123\";\n    spyOn(GlideRecord.prototype, \"addQuery\");\n    spyOn(GlideRecord.prototype, \"setLimit\");\n    spyOn(GlideRecord.prototype, \"query\");\n    spyOn(GlideRecord.prototype, \"canRead\").and.returnValue(true);\n    var result = service.getSPMInstanceData(companyId);\n    expect(GlideRecord.prototype.addQuery).toHaveBeenCalledWith(\"account\", \"123\");\n    expect(GlideRecord.prototype.setLimit).toHaveBeenCalledWith(1);\n    expect(GlideRecord.prototype.query).toHaveBeenCalled();\n    expect(GlideRecord.prototype.canRead).toHaveBeenCalled();\n    expect(result).toEqual({instanceName: \"\", instanceId: \"\", instanceRecordId: undefined});\n  });\n\n  it(\"should get linked SPM entity data correctly\", function() {\n    var companyId = \"123\";\n    var impactEntitySysId = \"456\";\n    var pageObj = {pageNo: 1, itemsPerPage: 10};\n    spyOn(GlideRecord.prototype, \"addEncodedQuery\");\n    spyOn(GlideRecord.prototype, \"orderBy\");\n    spyOn(GlideRecord.prototype, \"chooseWindow\");\n    spyOn(GlideRecord.prototype, \"query\");\n    spyOn(GlideRecord.prototype, \"canRead\").and.returnValue(true);\n    var result = service.getLinkedSPMEntityData(companyId, impactEntitySysId, pageObj);\n    expect(GlideRecord.prototype.addEncodedQuery).toHaveBeenCalledWith(\"impact_entity_record=456^company=123\");\n    expect(GlideRecord.prototype.orderBy).toHaveBeenCalledWith(\"spm_entity_record.name\");\n    expect(GlideRecord.prototype.chooseWindow).toHaveBeenCalledWith(0, 10);\n    expect(GlideRecord.prototype.query).toHaveBeenCalled();\n    expect(GlideRecord.prototype.canRead).toHaveBeenCalled();\n    expect(result).toEqual({result: [], totalRecord: 0});\n  });\n\n  it(\"should check if field mapping can be edited correctly\", function() {\n    var companySysId = \"123\";\n    spyOn(gs, \"hasRole\").and.returnValue(true);\n    spyOn(GlideRecord.prototype, \"addQuery\");\n    spyOn(GlideRecord.prototype, \"addQuery\");\n    spyOn(GlideRecord.prototype, \"addQuery\");\n    spyOn(GlideRecord.prototype, \"query\");\n    spyOn(GlideRecord.prototype, \"next\").and.returnValue(true);\n    spyOn(GlideRecord.prototype, \"canRead\").and.returnValue(true);\n    var result = service.canEditFieldMapping(companySysId);\n    expect(gs.hasRole).toHaveBeenCalledWith(\"x_snc_cusp.user\");\n    expect(GlideRecord.prototype.addQuery).toHaveBeenCalledWith(\"company\", \"123\");\n    expect(GlideRecord.prototype.addQuery).toHaveBeenCalledWith(\"user\", gs.getUserID());\n    expect(GlideRecord.prototype.addQuery).toHaveBeenCalledWith(\"contact_role.type\", \"internal\");\n    expect(GlideRecord.prototype.query).toHaveBeenCalled();\n    expect(GlideRecord.prototype.next).toHaveBeenCalled();\n    expect(GlideRecord.prototype.canRead).toHaveBeenCalled();\n    expect(result).toEqual(true);\n  });\n\n});", "role": "assistant"}}], "created": 1713922049, "id": "chatcmpl-9HLhxDclubEsgZ2jmKbQVWRO3wVTP", "model": "gpt-35-turbo", "object": "chat.completion", "system_fingerprint": null, "usage": {"completion_tokens": 2912, "prompt_tokens": 4026, "total_tokens": 6938}}
'''

In [39]:
parsed_data = parse_json(response)

AttributeError: 'str' object has no attribute 'items'

In [27]:
result_parsing = json.loads(json.dumps(response))

In [28]:
type(result_parsing)

str

In [ ]:
prompt_response,tokens_count,total_cost,model,status,status_code,error_message = model.gpt_35_turbo_16k_chat_completion()

In [18]:
print(response)

// Test case 1: Testing addition of positive numbers
describe("Addition", function() {
  it("should return the sum of three positive numbers", function() {
    var b = 2;
    var c = 3;
    var d = 4;
    var expectedSum = 9;
    var a = b + c + d;
    expect(a).toEqual(expectedSum);
  });
});

// Test case 2: Testing addition of negative numbers
describe("Addition", function() {
  it("should return the sum of three negative numbers", function() {
    var b = -2;
    var c = -3;
    var d = -4;
    var expectedSum = -9;
    var a = b + c + d;
    expect(a).toEqual(expectedSum);
  });
});

// Test case 3: Testing addition of positive and negative numbers
describe("Addition", function() {
  it("should return the sum of positive and negative numbers", function() {
    var b = 2;
    var c = -3;
    var d = 4;
    var expectedSum = 3;
    var a = b + c + d;
    expect(a).toEqual(expectedSum);
  });
});

// Test case 4: Testing addition of zero
describe("Addition", function() {
  it("should

##  Appendix

In [21]:
#!pip install langchain-core

In [22]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

examples = [
    {
        "question": "Who lived longer, Muhammad Ali or Alan Turing?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
"""
    },
    {
        "question": "When was the founder of craigslist born?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
"""
    },
    {
        "question": "Who was the maternal grandfather of George Washington?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
"""
    },
    {
        "question": "Are both the directors of Jaws and Casino Royale from the same country?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
"""
    }
]

In [23]:
example_prompt = PromptTemplate(
    input_variables=["question", "answer"], template="Question: {question}\n{answer}"
)


In [24]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

input_string = prompt.format(input="Who was the father of Mary Ball Washington?")

In [49]:
gpt_35_api_key = ullm_gpt35_key
gpt4_api_key =ullm_gpt4_key
url_link = "https://apidev.servicenow.com/ullm_dev/ullm-router/score"


def gpt_35_turbo_16k_chat_completion(prompt):
    #self.allowSelfSignedHttps(True)
    print(prompt)
    value = [{"role": "user", "content": quote(prompt)}]
    value_unquote = [{"role": "user", "content": prompt}]
    turbo_data = {
        "messages": value,
        'temperature': 0,
        'top_p': 0.1
    }

    turbo_data_unquote = {
        "messages": value_unquote,
        'temperature': 0,
        'top_p': 0.1
    }
    print(str(turbo_data_unquote))
    #print(str(turbo_data))
    turbo_body = str.encode(json.dumps(turbo_data))
    url = url_link
    api_key = gpt_35_api_key
    api_key_gpt4_32k = gpt4_api_key
    if not api_key:
        raise Exception("A key should be provided to invoke the endpoint")

    headers = {'Content-Type': 'application/json','api-key': api_key}

    #req = urllib.request.Request(url, turbo_body, headers)
    result =""
    prompt_response=""
    tokens_count = 0
    total_cost = 0.0
    model =""
    status =""
    status_code = 0
    error_message=""

    try:
        #req = urllib.request.Request(url, turbo_body, headers)
        response = requests.post(url=url,headers=headers,data=turbo_body)
        status_code = response.status_code
        response_json = response.json()
        print(response_json)
        prompt_response = response_json['choices'][0]['message']['content']
        print(prompt_response)
        tokens_count = response_json['usage']['total_tokens']
        total_cost = (int(response_json['usage']['prompt_tokens'])/1000)*0.0005 + (int(response_json['usage']['completion_tokens'])/1000)*0.0015
        total_cost = '{:.8f}'.format(total_cost)
        model = response_json['model']
        status = "success"

    except requests.exceptions.RequestException as error:  # This is the correct syntax
        
        status_code = error.errno
        status = "failed"
        print(error)
        #logger.info(error_message)
        raise(error)

    return prompt_response,tokens_count,total_cost,model,status,status_code,error_message

In [50]:
prompt_response,tokens_count,total_cost,model,status,status_code,error_message = gpt_35_turbo_16k_chat_completion(input_string)

Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali


Question: When was the founder of craigslist born?

Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952


Question: Who was the maternal grandfather of George Washington?

Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball W